# Experiment Collection #01 - Convex Solution

This notebook contains a convex solution to the problem tackled in experiment collection #01.

In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [ ]:
# Importing relevant libraries
import cvxpy as cp
import numpy as np

In [ ]:
from solara.constants import PROJECT_PATH
EXPERIMENT_NAME = "experiment_01_penalty_grid"
PLOT_DIR = PROJECT_PATH + "/figures/experiments/"
OUT_FORMAT = ".svg" # Output format of figures

# Loading data
# load_data = np.loadtxt(PROJECT_PATH + "/data/solar_trace_data_v2/load_5796.txt", delimiter=",")
# pv_data = np.loadtxt(PROJECT_PATH + "/data/solar_trace_data_v2/PV_5796.txt", delimiter=",")

load_data = np.loadtxt(PROJECT_PATH + "/data/ausgrid/processed/house2_combined_load.txt", delimiter=",")
pv_data = np.loadtxt(PROJECT_PATH + "/data/ausgrid/processed/house2_solar_gen.txt", delimiter=",")

In [ ]:
days = list(range(len(load_data)//24))

episodes_data = []

In [ ]:
from tqdm.notebook import tqdm

for day in tqdm(days):
    # Setting all the variables

    ## Given variables

    ### Basic
    T_u = 1 # Time slot duration
    T_h = 24 # Time horizon (hours)

    ### Grid
    pi_b = 0.14 #0.14 # Base price per unit of energy purchased ($/kWh)
    pi_d = 0.86 # Demand price penalty per unit of energy purchased with power demand exceeding Γ($/kWh)
    Gamma = 1.00 # np.percentile(load_data, 80) # Threshold above which the demand price is paid (kW)
    p_bar = 0.12 # Price per unit of energy sold at time t ($/kWh)

    ### Battery variables
    size = 10
    kWh_per_cell = 0.011284
    num_cells = size / kWh_per_cell

    nominal_voltage_c = 3.8793
    nominal_voltage_d = 3.5967
    u1 = 0.1920
    v1_bar = 0.0
    u2 = -0.4865
    v2_bar = kWh_per_cell * num_cells
    eta_d = 1 / 0.9  # taking reciprocal so that we don't divide by eta_d
    eta_c = 0.9942
    alpha_bar_d = (
        v2_bar * 1
    )  # the 1 indicates the maximum discharging C-rate
    alpha_bar_c = (
        v2_bar * 1
    )  # the 1 indicates the maximum charging C-rate


    # Given variables from data set
    num_timesteps = T_h
    start = 24*day
    power_load = load_data[start:start+num_timesteps] #np.random.randn(num_timesteps) # Load at time t (kW)
    power_solar = pv_data[start:start+num_timesteps] #np.random.randn(num_timesteps) # Power generated by solar panels at timet(kW)

    # Variables that are being optimised over
    power_direct = cp.Variable(num_timesteps) # Power flowing directly from PV and grid to meet the load or be sold at time t (kW) (P_dir)
    power_charge = cp.Variable(num_timesteps) # Power used to charge the ESD at time t (kW) (P_c)
    power_discharge = cp.Variable(num_timesteps) # Power from the ESD at time t (kW) (P_d)
    power_grid = cp.Variable(num_timesteps) # Power drawn from the grid at time t (kW) (P_g)
    power_sell = cp.Variable(num_timesteps) # Power sold to the grid at timet(kW) (P_sell)
    power_over_thres = cp.Variable(num_timesteps) #  Purchased power that exceeds Γ at time t (not in notation table) (P_over)

    # Implicitly defined variable (not in paper in "given" or "optimized over" set of variables)
    energy_battery = cp.Variable(num_timesteps+1) # the  energy  content  of  the  ESD  at  the  beginning  of  interval t (E_ESD)
    
    base_constraints = [
    0 <= power_grid, # from Equation (13)
    0 <= power_direct,
    0 <= power_sell,
    0 <= power_charge, # Eq (18)
    0 <= power_discharge, # Eq  (19)

    # Power flow
    power_direct + power_discharge == power_load + power_sell, # from Equation (14)
    0 <= power_charge + power_direct, # Eq (17)
    power_charge + power_direct <= power_solar + power_grid, # Eq (17)
    ]

    grid_constraints = [
        0 <= power_over_thres,
        power_grid - Gamma <= power_over_thres, # Eq (24)
        power_sell == 0, # stopping selling to the grid
        ]

    battery_constraints = [
        energy_battery[0] == 0,
        energy_battery[1:] == energy_battery[:-1] + eta_c*power_charge*T_u - eta_d * power_discharge * T_u,
        energy_battery >= 0,
        power_discharge <= alpha_bar_d,
        power_charge <= alpha_bar_c, #equation (5)
        u1 * ((power_discharge)/nominal_voltage_d) + v1_bar <= energy_battery[1:], # equation (4)
        u2 * ((power_charge)/nominal_voltage_c) + v2_bar >= energy_battery[1:], # equation (4)

    ]

    constraints = base_constraints + battery_constraints + grid_constraints
    
    objective = cp.Minimize(cp.sum(pi_b*power_grid + pi_d*power_over_thres - cp.multiply(p_bar,power_sell)))
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve(verbose=False)
    
    charging_power = power_charge.value - power_discharge.value
    
    episode_data = {
        'load': power_load,
        'pv_gen': power_solar,
        'net_load': power_grid.value,
        'battery_cont': energy_battery.value,
        'charging_power': charging_power,
        'cost': pi_b*power_grid.value + pi_d*power_over_thres.value,
        'price_threshold': np.ones(25) * Gamma,
        'actions': charging_power / 10,
        'rewards': - (pi_b*power_grid.value + pi_d*power_over_thres.value),
        'power_diff': np.zeros(24),
    }
    
    episodes_data.append(episode_data)

In [ ]:
ep_costs = []
for ep in episodes_data:
    ep_costs.append(np.sum(ep["cost"]))
np.mean(ep_costs)

In [ ]:
import matplotlib.pyplot as plt

# Plotting configuration
POLICY_PLOT_CONF = {
    "selected_keys": ['load','pv_gen','energy_cont','net_load',
              'charging_power','cost','price_threshold', #'battery_cont',
              'actions'],
    "y_min":-1.3,
    "y_max":1.4,
    "show_grid":False,
}

solara.plot.pyplot.plot_episode(episode_data,title=None, **POLICY_PLOT_CONF)
plt.savefig(fname=PLOT_DIR + EXPERIMENT_NAME + "_plot_09_convex_solution" + OUT_FORMAT, bbox_inches='tight')
plt.show()

In [ ]:
import solara.envs.components.solar
import solara.envs.components.load
import solara.envs.components.grid
import solara.envs.components.battery
import solara.envs.battery_control
import solara.utils.logging
from solara.constants import PROJECT_PATH


def battery_env_creator(env_config=None):
    """Create a battery control environment."""
    
    PV_DATA_PATH = PROJECT_PATH + "/data/solar_trace_data/PV_5796.txt"
    LOAD_DATA_PATH = PROJECT_PATH + "/data/solar_trace_data/load_5796.txt"

    # Setting up components of environment
    battery_model = solara.envs.components.battery.LithiumIonBattery(size=10, 
                                                                     chemistry="NMC", 
                                                                     time_step_len=1)
    pv_model = solara.envs.components.solar.DataPV(data_path=PV_DATA_PATH,
                                                   fixed_sample_num=12)
    load_model = solara.envs.components.load.DataLoad(data_path=LOAD_DATA_PATH,
                                                      fixed_sample_num=12)
    grid_model = solara.envs.components.grid.PeakGrid(peak_threshold=1.0)

    # Fixing load and PV trace to single sample
    episode_num = 12
    load_model.fix_start(episode_num)
    pv_model.fix_start(episode_num)

    env = solara.envs.battery_control.BatteryControlEnv(
        battery = battery_model,
        solar = pv_model,
        grid = grid_model,
        load = load_model,
        infeasible_control_penalty=True,
        grid_charging=True,
        logging_level = "WARNING",
    )
    
    return env

env = battery_env_creator()

solara.plot.widgets.InteractiveEpisodes([episode_data], 
                                        initial_visibility=initial_visibility, 
                                        manual_mode=True, 
                                        manual_start_actions=episode_data["actions"],
                                        env=env)

In [ ]:
episode_data["actions"][0]